This notebook is used to merge crawled data from different sources, preprocess them and write to to DB before indexing process

In [ ]:
# !pip install polyglot
# !pip install polyglot
# !pip install PyICU
# !pip install pycld2
# !pip install langdetect
# !pip install swifter

In [ ]:
import duckdb
import pandas as pd
import swifter
from langdetect import detect
from polyglot.detect import Detector

In [ ]:
conn_existing = duckdb.connect('crawlerDb.duckdb')
conn_new = duckdb.connect('crawler_v4.db')

In [ ]:
df_existing = conn_existing.execute("select * from urlsDB").df()
df_new = conn_new.execute("select * from urlsDB").df()

In [ ]:
# to filter out duplicate URLs
def normalise_url(url):
    if url.startswith("https://"):
        return url[8:]
    elif url.startswith("http://"):
        return url[7:]
    # remove query parameters
    if '?' in url:
        url = url.split('?')[0]
    # remove trailing slashes
    if url.endswith('/'):
        url = url[:-1]
    return url

In [ ]:
df_existing['normalised_url'] = df_existing['url'].swifter.apply(normalise_url)
df_new['normalised_url'] = df_new['url'].swifter.apply(normalise_url)

In [ ]:
# remove duplicates
print(f'len(df_new): {len(df_new)}')
df_new = df_new[~df_new['normalised_url'].isin(df_existing['normalised_url'])]
print(f'len(df_new) after removing existing: {len(df_new)}')
# remove duplicates within df_new
df_new = df_new.drop_duplicates(subset=['normalised_url'])
print(f'len(df_new) after removing duplicates: {len(df_new)}')

In [ ]:
df_new.drop(columns=['normalised_url'], inplace=True)
df_existing.drop(columns=['normalised_url'], inplace=True)

## Language Deetection:
Detecting if a text is in English isn't quite easy. Some of the existing python packages struggle with either very short or long texts. To be lenient, we try detetcting the english language from libraries langdetect and polygot. If either of them says the language of the text is english with a confidence of atleast 15% we accept, else we reject it

In [ ]:
def is_english_langdetect(text):
    if len(text) > 2:
        try:
            return detect(text) == 'en'
        except:
            print(f"Error detecting language for text: {text[:100]}...")  
            return False
    return False

def is_english_polyglot(text):
    try:
        detector = Detector(text)
        for language in detector.languages:
            if language.code == 'en' and language.confidence > 0.15:
                return True
        return False
    except Exception as e:
        print(f"Error detecting language for text: {text[:100]}... Error: {e}") 
        return False


In [ ]:
df_new['is_english_langdetect'] = df_new['text'].swifter.apply(is_english_langdetect)
df_new['is_english_polyglot'] = df_new['text'].swifter.apply(is_english_polyglot)

In [ ]:
df_new.is_english_langdetect.value_counts(), df_new.is_english_polyglot.value_counts()

In [ ]:
df_new_english = df_new[(df_new.is_english_langdetect == True) | (df_new.is_english_polyglot == True)]


In [ ]:
df_english = df_new_english.drop(columns=['is_english_langdetect', 'is_english_polyglot'])
df_english = df_english.drop_duplicates(subset=['url'])


In [ ]:
max_id = df_existing['id'].max()
df_english['id'] = range(max_id + 1, max_id + 1 + len(df_english))


In [ ]:
# change lastFetch to unix timestamp
df_english['lastFetch'] = pd.to_datetime(df_english['lastFetch']).astype('int64') 
# rename columns
df_english = df_english.rename(columns={
    'parentUrl': 'incoming',
}, inplace=True)
# remove columns that are not in df_existing
df_english = df_english[df_existing.columns]
df_english

In [ ]:
concat_df = pd.concat([df_existing, df_english], ignore_index=True)


In [ ]:
# delete the old table and create a new one
conn_existing.execute("DROP TABLE IF EXISTS urlsDB").df()

In [ ]:
conn_existing.execute("CREATE TABLE urlsDB AS SELECT * FROM concat_df").df()

In [ ]:
conn_existing.close()
conn_new.close()